В стандартной установке может отсутствовать часть модулей, необходимых для работы, их нужно установить  
Команда установки "pip install имя_модуля" вводится в этом же окне.  
После установки необходимо перезапустить ядро выбрав в меню сверху Kernel-Restart. 

In [1]:
import pandas as pd #Для работы с таблицами данных (дата фреймы)
import requests #Для запросов к серверу
import json #Для обработки ответов сервера

# Выгрузка данных по акции Сбербанка

Сперва зададим изначально необходимые значения  
Сформируем базовый заспрос и пустой дата фрейм с именами колонок

In [2]:
print("Hello world")

Hello world


In [4]:
# В базовом запросе мы укажем:
# engines/stok (фондовый рынок), markets/shares - рынок акций, boards/TQBR - режим торгов для акции сбербанка TQBR
# Базовый запрос поместим в переменную base_url
base_url = "http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities.json"
response = requests.get(base_url) # Получим ответ от сервера
result = json.loads(response.text) # Расшифровываем ответ от сервера
col_name = result['history']['columns'] # Задаем имена колонок извлекая данные из ответа сервера
data_shares = pd.DataFrame(columns = col_name) #

In [6]:
data_shares

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR,TRADE_SESSION_DATE


In [7]:
data_shares  # Можно посмотреть что получилось

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR,TRADE_SESSION_DATE


In [12]:
url_share = 'http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities/SBER.json' #Первая страница данных
# Получаем данные
response = requests.get(url_share)
result = json.loads(response.text)
resp_date = result['history']['data']
#Извлеченные данные добавляем в начало дата фрейма
data_shares  = pd.DataFrame(resp_date, columns = col_name)
# Узнаем количество полученных строк, если их меньше 100, то мы получили все данные
a = len(resp_date)
a

100

In [14]:
# resp_date

Если же значение а равно 100, то получены не все данные по выбранной бумаге.  
В этом случае запускаем в цикле переборку остальных страниц.  
Цикл будет работать до тех пор, пока мы не получим количество строк меньше, чем 100.  
Для этого каждый раз на предпоследнем шаге цикла переопределяем переменную.

In [9]:
b = 100
while a == 100:
    url_opt = '?start=' + str(b)
    url_next_page  = url_share + url_opt
    response = requests.get(url_next_page)
    result = json.loads(response.text)
    resp_date = result['history']['data']
    data_next_page = pd.DataFrame(resp_date, columns = col_name)
    data_shares = pd.concat([data_shares, data_next_page], ignore_index=True)
    a = len(resp_date)
    b = b + 100

/tmp/ipykernel_13120/2378968065.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_shares = pd.concat([data_shares, data_next_page], ignore_index=True)


In [10]:
b

3100

Судя по переменной b, у нас должно быть не менее 2400 строк, проверим

In [15]:
data_shares

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR,TRADE_SESSION_DATE
0,TQBR,2013-03-25,Сбербанк,SBER,140,59340002.8,96.00,96.00,101.14,98.66,...,99.99,99.99,8.572434e+09,8.572434e+09,8.572434e+09,None,3,SUR,NaN,None
1,TQBR,2013-03-26,Сбербанк,SBER,183,126030358.8,98.58,97.08,99.31,97.13,...,97.82,97.82,1.221847e+10,1.221847e+10,1.221847e+10,None,3,SUR,-1.61,None
2,TQBR,2013-03-27,Сбербанк,SBER,176,121835900.2,97.90,95.39,98.00,96.73,...,96.45,96.45,1.251981e+10,1.251981e+10,1.251981e+10,None,3,SUR,-0.46,None
3,TQBR,2013-03-28,Сбербанк,SBER,299,192469794.9,96.38,95.72,98.66,98.64,...,97.74,97.74,1.257281e+10,1.257281e+10,1.257281e+10,None,3,SUR,1.90,None
4,TQBR,2013-03-29,Сбербанк,SBER,121,77268860.1,98.60,98.32,99.09,98.83,...,98.77,98.77,2.611474e+09,2.611474e+09,2.611474e+09,None,3,SUR,0.17,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,TQBR,2013-08-08,Сбербанк,SBER,2401,890408055.5,95.00,93.32,95.15,94.00,...,94.15,94.15,5.923967e+09,5.923967e+09,5.923967e+09,None,3,SUR,-0.41,None
96,TQBR,2013-08-09,Сбербанк,SBER,2514,860309410.9,94.48,93.95,95.74,95.66,...,94.86,94.86,6.528187e+09,6.528187e+09,6.528187e+09,None,3,SUR,1.71,None
97,TQBR,2013-08-12,Сбербанк,SBER,2451,816471222.0,95.80,94.78,96.40,96.21,...,95.48,95.48,5.652812e+09,5.652812e+09,5.652812e+09,None,3,SUR,0.54,None
98,TQBR,2013-08-13,Сбербанк,SBER,2194,917885512.5,95.93,95.93,97.93,97.20,...,97.34,97.34,7.378259e+09,7.378259e+09,7.378259e+09,None,3,SUR,1.23,None


Мы получили 2405 строк, с датами с 25 марта 2013 г. по предыдущую торговую дату

# Облигации

Данные по облигациям собираются аналогичным образом, изменяется лишь базовый запрос.  
https://iss.moex.com/iss/history/engines/stock/markets/bonds/boards/TQCB/securities  
Как видно из базового запроса, у нас изменился параметр после markets - shares поменяли на bonds и значение режима идентификатора торгов с TQBR на TQCB
Найти название нужной облигаии можно на сайте московской биржи в разделе "Биржевая информация". Выбрав нужный раздел и бумагу, мы увидим необходимую информацию

Возьмем облигацию РЖД-32 обл. Ее код RU000A0JSGV0, идентификатор режима торгов TQCB  
Кроме этого, можно получить информацию об инструменте и через API, указав в запросе код бумаги
https://iss.moex.com/iss/securities/RU000A0JSGV0

In [16]:
base_url = "http://iss.moex.com/iss/history/engines/stock/markets/bonds/boards/TQCB/securities.json" #Задаем базовый адрес запроса для облигаций
response = requests.get(base_url) #Получим ответ от сервера
result = json.loads(response.text)
col_name = result['history']['columns'] #Задаем имена колонок извлекая данные из ответа сервера
data_bonds = pd.DataFrame(columns = col_name)

Проверим полученые колонки

In [17]:
data_bonds

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,LOW,HIGH,CLOSE,LEGALCLOSEPRICE,...,YIELDLASTCOUPON,OFFERDATE,FACEUNIT,TRADINGSESSION,CALLOPTIONDATE,CALLOPTIONYIELD,CALLOPTIONDURATION,PUTOPTIONDATE,DATEYIELDFROMISSUER,TRADE_SESSION_DATE


In [18]:
url_bonds = 'http://iss.moex.com/iss/history/engines/stock/markets/bonds/boards/TQCB/securities/RU000A0JSGV0.json' #Первая страница данных
# Получаем данные
response = requests.get(url_bonds)
result = json.loads(response.text)
resp_date = result['history']['data']
#Извлеченные данные добавляем в начальный дата фрейм
data_bonds  = pd.DataFrame(resp_date, columns = col_name)
# Узнаем количество полученных строк, если их меньше 100, то мы получили все данные
a = len(resp_date)

Проверим первые полученные данные

In [19]:
data_bonds

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,LOW,HIGH,CLOSE,LEGALCLOSEPRICE,...,YIELDLASTCOUPON,OFFERDATE,FACEUNIT,TRADINGSESSION,CALLOPTIONDATE,CALLOPTIONYIELD,CALLOPTIONDURATION,PUTOPTIONDATE,DATEYIELDFROMISSUER,TRADE_SESSION_DATE
0,TQCB,2020-05-25,РЖД-32 обл,RU000A0JSGV0,28,10197348.3,99.60,99.83,99.79,99.71,...,None,2022-07-13,RUB,3,None,None,None,None,None,None
1,TQCB,2020-05-26,РЖД-32 обл,RU000A0JSGV0,58,1241219.7,99.72,99.99,99.78,99.78,...,None,2022-07-13,RUB,3,None,None,None,None,None,None
2,TQCB,2020-05-27,РЖД-32 обл,RU000A0JSGV0,57,1918925.5,99.65,99.83,99.80,99.80,...,None,2022-07-13,RUB,3,None,None,None,None,None,None
3,TQCB,2020-05-28,РЖД-32 обл,RU000A0JSGV0,84,148670.0,99.68,99.85,99.71,99.72,...,None,2022-07-13,RUB,3,None,None,None,None,None,None
4,TQCB,2020-05-29,РЖД-32 обл,RU000A0JSGV0,36,286536.0,99.72,99.84,99.83,99.83,...,None,2022-07-13,RUB,3,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,TQCB,2020-10-08,РЖД-32 обл,RU000A0JSGV0,13,21968.0,99.75,99.90,99.75,99.75,...,None,2022-07-13,RUB,3,None,None,None,None,None,None
96,TQCB,2020-10-09,РЖД-32 обл,RU000A0JSGV0,27,113788.1,99.75,100.00,99.83,99.83,...,None,2022-07-13,RUB,3,None,None,None,None,None,None
97,TQCB,2020-10-12,РЖД-32 обл,RU000A0JSGV0,31,380901.6,99.70,99.94,99.81,99.81,...,None,2022-07-13,RUB,3,None,None,None,None,None,None
98,TQCB,2020-10-13,РЖД-32 обл,RU000A0JSGV0,56,209480.7,99.53,99.89,99.86,99.86,...,None,2022-07-13,RUB,3,None,None,None,None,None,None


На сайте мосбиржи для этой бумаги указана дата начала торгов 20.07.2012, однако мы получили данные только с 25 мая 2020 года. 
Открыв на сайте мос биржи на странице этой бумаги вкладку "Режимы торгов", можно увидеть, что в указанном режиме торгов бумага торгуется как раз с 25.05.2020, однако никакой информации за более ранние даты даже по другим режимам торгов информация через API и на сайте биржи нет

Как и в примере с акцией Сбера, собираем в цикле данные по облигации РЖД

In [20]:
b = 100
while a == 100:
    url_opt = '?start=' + str(b)
    url_next_page  = url_bonds + url_opt
    response = requests.get(url_next_page)
    result = json.loads(response.text)
    resp_date = result['history']['data']
    data_next_page = pd.DataFrame(resp_date, columns = col_name)
    data_bonds  = pd.concat([data_bonds, data_next_page], ignore_index=True)
    a = len(resp_date)
    b = b + 100

/tmp/ipykernel_13120/2763533386.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_bonds  = pd.concat([data_bonds, data_next_page], ignore_index=True)


Проверяем, что получилось

In [21]:
data_bonds

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,LOW,HIGH,CLOSE,LEGALCLOSEPRICE,...,YIELDLASTCOUPON,OFFERDATE,FACEUNIT,TRADINGSESSION,CALLOPTIONDATE,CALLOPTIONYIELD,CALLOPTIONDURATION,PUTOPTIONDATE,DATEYIELDFROMISSUER,TRADE_SESSION_DATE
0,TQCB,2020-05-25,РЖД-32 обл,RU000A0JSGV0,28,10197348.3,99.60,99.83,99.79,99.71,...,None,2022-07-13,RUB,3,None,None,None,None,None,None
1,TQCB,2020-05-26,РЖД-32 обл,RU000A0JSGV0,58,1241219.7,99.72,99.99,99.78,99.78,...,None,2022-07-13,RUB,3,None,None,None,None,None,None
2,TQCB,2020-05-27,РЖД-32 обл,RU000A0JSGV0,57,1918925.5,99.65,99.83,99.80,99.80,...,None,2022-07-13,RUB,3,None,None,None,None,None,None
3,TQCB,2020-05-28,РЖД-32 обл,RU000A0JSGV0,84,148670.0,99.68,99.85,99.71,99.72,...,None,2022-07-13,RUB,3,None,None,None,None,None,None
4,TQCB,2020-05-29,РЖД-32 обл,RU000A0JSGV0,36,286536.0,99.72,99.84,99.83,99.83,...,None,2022-07-13,RUB,3,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1236,TQCB,2025-04-07,РЖД-32 обл,RU000A0JSGV0,56,230033.6,88.25,89.95,89.95,89.35,...,None,None,RUB,3,None,None,None,None,2026-07-03,2025-04-07
1237,TQCB,2025-04-08,РЖД-32 обл,RU000A0JSGV0,67,217332.2,88.10,91.12,89.05,89.23,...,None,None,RUB,3,None,None,None,None,2026-07-03,2025-04-08
1238,TQCB,2025-04-09,РЖД-32 обл,RU000A0JSGV0,53,179404.5,89.09,90.90,90.49,90.49,...,None,None,RUB,3,None,None,None,None,2026-07-03,2025-04-09
1239,TQCB,2025-04-10,РЖД-32 обл,RU000A0JSGV0,73,154397.1,88.01,91.23,88.82,88.82,...,None,None,RUB,3,None,None,None,None,2026-07-03,2025-04-10


Как мы видим, были выгружены данные с 25 мая 2020 г. по предыдущую торговую дату

# Другие инструменты

Попробуем выгрузить аналогичную информацию по другим инструментам мосбиржи  
Например индекс мосбиржи IMOEX  
Сперва получим информацию о бумаге: http://iss.moex.com/iss/securities/IMOEX  
Получим следующую базовую информацию:  
engine - stock  
market  - index  
boardid - SNDX  
Составляем базовый запрос:
http://iss.moex.com/iss/history/engines/stock/markets/bonds/boards/SNDX/securities.json  
Запрос для выбранной бумаги: 
http://iss.moex.com/iss/history/engines/stock/markets/index/boards/SNDX/securities/imoex.json  
Как видно из ответа, по индексу IMOEX доступны данные начиная с 22 сентября 1997 года  
Скачаем данные аналогично другим инструментам

In [22]:
base_url = "http://iss.moex.com/iss/history/engines/stock/markets/index/boards/SNDX/securities.json" #Задаем базовый адрес запроса для облигаций
response = requests.get(base_url) #Получим ответ от сервера
result = json.loads(response.text)
col_name = result['history']['columns'] #Задаем имена колонок извлекая данные из ответа сервера
data_index = pd.DataFrame(columns = col_name)

In [23]:
data_index

,BOARDID,SECID,TRADEDATE,SHORTNAME,NAME,CLOSE,OPEN,HIGH,LOW,VALUE,DURATION,YIELD,DECIMALS,CAPITALIZATION,CURRENCYID,DIVISOR,TRADINGSESSION,VOLUME


In [24]:
url_index = 'http://iss.moex.com/iss/history/engines/stock/markets/index/boards/SNDX/securities/imoex.json' #Первая страница данных
# Получаем данные
response = requests.get(url_index)
result = json.loads(response.text)
resp_date = result['history']['data']
#Извлеченные данные добавляем в начальный дата фрейм
data_index  = pd.DataFrame(resp_date, columns = col_name)
# Узнаем количество полученных строк, если их меньше 100, то мы получили все данные
a = len(resp_date)

In [25]:
data_index

,BOARDID,SECID,TRADEDATE,SHORTNAME,NAME,CLOSE,OPEN,HIGH,LOW,VALUE,DURATION,YIELD,DECIMALS,CAPITALIZATION,CURRENCYID,DIVISOR,TRADINGSESSION,VOLUME
0,SNDX,IMOEX,1997-09-22,Индекс МосБиржи,Индекс МосБиржи,100.00,None,None,None,None,None,None,2,None,RUB,None,3,None
1,SNDX,IMOEX,1997-09-23,Индекс МосБиржи,Индекс МосБиржи,100.67,None,None,None,None,None,None,2,None,RUB,None,3,None
2,SNDX,IMOEX,1997-09-24,Индекс МосБиржи,Индекс МосБиржи,99.94,None,None,None,None,None,None,2,None,RUB,None,3,None
3,SNDX,IMOEX,1997-09-25,Индекс МосБиржи,Индекс МосБиржи,99.46,None,None,None,None,None,None,2,None,RUB,None,3,None
4,SNDX,IMOEX,1997-09-26,Индекс МосБиржи,Индекс МосБиржи,98.87,None,None,None,None,None,None,2,None,RUB,None,3,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,SNDX,IMOEX,1998-02-11,Индекс МосБиржи,Индекс МосБиржи,72.15,None,None,None,None,None,None,2,None,RUB,None,3,None
96,SNDX,IMOEX,1998-02-12,Индекс МосБиржи,Индекс МосБиржи,70.63,None,None,None,None,None,None,2,None,RUB,None,3,None
97,SNDX,IMOEX,1998-02-13,Индекс МосБиржи,Индекс МосБиржи,69.60,None,None,None,None,None,None,2,None,RUB,None,3,None
98,SNDX,IMOEX,1998-02-16,Индекс МосБиржи,Индекс МосБиржи,70.32,None,None,None,None,None,None,2,None,RUB,None,3,None


In [26]:
b = 100
while a == 100:
    url_opt = '?start=' + str(b)
    url_next_page  = (url_index + url_opt)
    response = requests.get(url_next_page)
    result = json.loads(response.text)
    resp_date = result['history']['data']
    data_next_page = pd.DataFrame(resp_date, columns = col_name)
    data_index  = pd.concat([data_index, data_next_page], ignore_index=True)
    a = len(resp_date)
    b = b + 100

/tmp/ipykernel_13120/249888975.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_index  = pd.concat([data_index, data_next_page], ignore_index=True)


In [27]:
data_index

,BOARDID,SECID,TRADEDATE,SHORTNAME,NAME,CLOSE,OPEN,HIGH,LOW,VALUE,DURATION,YIELD,DECIMALS,CAPITALIZATION,CURRENCYID,DIVISOR,TRADINGSESSION,VOLUME
0,SNDX,IMOEX,1997-09-22,Индекс МосБиржи,Индекс МосБиржи,100.00,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,RUB,NaN,3,None
1,SNDX,IMOEX,1997-09-23,Индекс МосБиржи,Индекс МосБиржи,100.67,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,RUB,NaN,3,None
2,SNDX,IMOEX,1997-09-24,Индекс МосБиржи,Индекс МосБиржи,99.94,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,RUB,NaN,3,None
3,SNDX,IMOEX,1997-09-25,Индекс МосБиржи,Индекс МосБиржи,99.46,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,RUB,NaN,3,None
4,SNDX,IMOEX,1997-09-26,Индекс МосБиржи,Индекс МосБиржи,98.87,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,RUB,NaN,3,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6887,SNDX,IMOEX,2025-04-07,Индекс МосБиржи,Индекс МосБиржи,2729.89,2648.09,2791.28,2646.70,1.656315e+11,0.0,0.0,2,6.074632e+12,RUB,2.225232e+09,3,None
6888,SNDX,IMOEX,2025-04-08,Индекс МосБиржи,Индекс МосБиржи,2746.41,2775.78,2780.46,2727.98,7.414724e+10,0.0,0.0,2,6.111401e+12,RUB,2.225232e+09,3,None
6889,SNDX,IMOEX,2025-04-09,Индекс МосБиржи,Индекс МосБиржи,2650.69,2689.77,2707.15,2598.85,1.237969e+11,0.0,0.0,2,5.898403e+12,RUB,2.225232e+09,3,None
6890,SNDX,IMOEX,2025-04-10,Индекс МосБиржи,Индекс МосБиржи,2764.28,2790.31,2801.19,2742.48,9.068518e+10,0.0,0.0,2,6.151159e+12,RUB,2.225232e+09,3,None


В итоге мы за несколько секунд получили 6254 строки или данные по инструменту с 22 сентября 1997 г. по последнюю торговую дату

Все полученные данные можно сохранить в файл формата csv для последующей обработки удобным методом

In [28]:
data_shares.to_csv('SBER_22.csv', encoding='cp1251')
data_bonds.to_csv('RZD_22.csv', encoding='cp1251')
data_index.to_csv('IMOEX_22.csv', encoding='cp1251', sep=';')

Синтаксис сохранения в файл простой  
имя_набора данных.to_csv('имя файла с расширением', encoding='кодировка_файла', sep='разделитель')  
Параметр кодировки необязательный, но при работе с кирилицей лучше использовать его  
Параметр разделителя - по умолчанию это ','. Если использовать sep=';', то данные при открытии csv файла Excel'ем будут уже разбиты на столбы, иначе вы получите данные одной строки в одном столбце, значения в котором будут разделены запятой  
Если необходимо сохранить файл в определенное место на компьютере, а не в рабочей папке проект, то имя файла указывается в следующем формате: data.to_csv(r'D:\имя_папки\имя_файла.csv', encoding='cp1251')'
